In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of src.main failed: Traceback (most recent call last):
  File "C:\Users\merti\anaconda3\envs\uitest\Lib\site-packages\IPython\extensions\autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\merti\anaconda3\envs\uitest\Lib\site-packages\IPython\extensions\autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "C:\Users\merti\anaconda3\envs\uitest\Lib\importlib\__init__.py", line 131, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 866, in _exec
  File "<frozen importlib._bootstrap_external>", line 995, in exec_module
  File "<frozen importlib._bootstrap>", line 488, in _call_with_frames_removed
  File "C:\Users\merti\PycharmProjects\cadenza-playwright-llm\src\main.py", line 5, in <module>
    from src.ui_tests.test_generation import generate_code, generate_code_on_cluster
ImportError: cannot import name 'generate_code_on_cluster' from 'src.ui

In [30]:
import sqlite3
from bs4 import BeautifulSoup

from src.main import main
from src.utils.helpers import truncate_text, clean_string
from src.utils.logger import setup_logger
from src.llm.access_2_cluster import Access2Cluster
from src.data.html_processor import extract_html_info

logger = setup_logger(__name__, level='DEBUG') # Change to 'INFO' for less verbosity

# Load Data

In [31]:
# Connect to the database
conn = sqlite3.connect('../data/raw/playwright_script.db')
cursor = conn.cursor()

res = cursor.execute("SELECT * FROM tests")
items = res.fetchall()

print("There are {} data.".format(len(items)))

There are 100 data.


In [32]:
# Check the first item
items[0]

('1.1',
 '[1.1] Öffne die Arbeitsmappe "Übersicht Messstellen" im Ordner "Gewässergüte".',
 '[1.1] Expected result: Die Arbeitsmappe wird geöffnet, der Analysekontext ist nicht sichtbar.',
 '.\\html\\1_1.html',
 '.\\screenshot\\1_1.png',
 '.\\test_script\\1_1.spec.ts')

In [33]:
items[-1]

('30.4',
 '[30.1] Öffne den Link "Übersicht Messstellen" im Ordner "Gewässergüte". [30.2] Öffne die Karten-Sicht "Messstellenkarte" über die Werkzeugliste der Arbeitsmappe. [30.3] Klicke auf den 3-Punkte-Button innerhalb der Karten-Sicht "Messstellenlage und Wasserschutzgebiete eingefärbt nach Landkreisen" [30.4] Klicke auf "Duplizieren"',
 '[30.4] Expected result: Das Kontextmenü schließt sich. Die Sicht ist dupliziert. Die duplizierte Sicht erscheint neben der Original-Sicht. Beide Sichten teilen sich in gleichen Maßen den Platz der Original-Sicht.',
 '.\\html\\30_4.html',
 '.\\screenshot\\30_4.png',
 '.\\test_script\\30_4.spec.ts')

# Approach

Use a Pre-trained LLM:
* GPT-3, GPT-3.5, or a lighter version like GPT-2 (well-suited for text generation tasks)
* Fine-tuning or adapting for specific tasks later possible

Steps:
1. ✅ HTML Processing: Extract relevant information from the HTML file.
    * Use **BeautifulSoup** or lxml in Python to parse and extract information from the HTML file. ➡️ see src.data.html_processor.py
2. ✅ Image Processing: Extract relevant information from the image:
    * Use image-to-text models like **Tesseracts** or pytesseract (OCR libraries) to extract text from the image. ➡️ see src.data.image_processor.py
    * Use OpenCV or PIL (Pillow) in Python to process the image and extract relevant information.
3. Summarize the image and HTML information and the prompt from the playwright test code using T5 model. (optional)
4. ✅ Python Processing: Parse the given playwright test code for previous step as a precondition. ➡️ see src.data.python_processor.py
5. ✅ Combine the extracted information from the HTML and the image with the prompt for the language model. ➡️ see src.data.input_combiner.py
6. ✅ Pass the combined input to the language model for generating the UI test code. ➡️ see src.ui_tests.test_generation.py

➡️ Run locally via notebook or script using the src.main.py

# HTML Processing

In [5]:
# load example file
html_path = './html/0_1.html'

Option 1: Parse all HTML content by extracting the text from it using BeautifulSoup:

In [6]:
def parse_html(html_path: str, max_length: int = 200) -> str:
    """ Parse the HTML content from a file using BeautifulSoup. It extracts the text content and truncates it to the given maximum length.

    :param max_length: The maximum length of the text.
    :param html_path: The path to the HTML file.
    :return: The text content of the HTML file.
    """
    # Load HTML content from a file
    with open(html_path, "r") as file:
        html_content = file.read()

    # Parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')

    # Get the text content
    html_text = soup.get_text(strip=True)

    # Truncate the text to the maximum length
    html_text = truncate_text(html_text, max_length=max_length)

    logger.debug(f"HTML content parsed successfully. - Lines of Code: {len(html_text.splitlines())}")

    return html_text

In [7]:
html_text = parse_html(html_path, max_length=2000)
print(html_text)

2024-07-02 10:00:52 [__main__:21] [DEBUG] >>>> HTML content parsed successfully. - Lines of Code: 3
Startseite - disy CadenzaAchtung!Ihr Browser unterstÃ¼tzt kein JavaScript oder JavaScript wurde in Ihrem Browser deaktiviert.Bitte verwenden Sie einen Browser, der JavaScript unterstÃ¼tzt, oder aktivieren Sie JavaScript in Ihrem Browser. Ohne aktiviertes JavaScript ist die Anwendung nicht nutzbar.OfflineVerbinden â€¦Zum Navigatorbaum springenZum Hauptbereich springenStartseiteKartedisy Cadenza[{"printName":"Lernmodule â€“ Tutorials und mehr","url":"/help-learning/index.html","targetFrame":"_blank","id":"help","type":"help","webApplication":false},{"printName":"Hilfe","url":"/help/","targetFrame":"_blank","id":"help","type":"help","webApplication":false},{"printName":"Hilfe zu Classic","url":"/help-classic/","targetFrame":"_blank","id":"help-classic","type":"help-classic","webApplication":false}]Admin[{"printName":"Profil","url":"/pages/access/userprofile.xhtml","targetFrame":"_self","id"

Option 2: Extract only Elements (input fields, buttons, links) from the HTML file using BeautifulSoup.

In [8]:
html_elements = extract_html_info(html_path, concat_mod='all')
print(html_elements)

Buttons: 
Button ids: navigationTrigger, workbook-create, RDxYr2vFytOijWjelj7P1, 
Button classes: button button-icon button-borderless, button workbook-create button-icon, button navigation-menu button-icon
Inputs: 
Input classes: select2-search__field, 
Input aria-labels: Suchen nach …, 
Input types: search, 
Input placeholders: Suchen nach …
Links: 
Link ids: skip-to-navigator, skip-to-content, home, 
Link classes: button button-primary, button button-primary, button button-icon button-borderless



# (Local) Pipeline: LLM UI Test Generation
Using smaller GPT-2 model

1. First, extract the relevant information from the database for a specific test to be predicted:

In [9]:
def get_previous_id(id):
    test, step = map(int, id.split('.'))
    if step > 1:
        previous_id = f"{test}.{step - 1}"
    else:
        print(f"Test {test} has no more previous step. No context available.")

    return previous_id

In [10]:
def fetch_relevant_items(db_file, current_id):
    # Connect to SQLite database
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Get the previous ID
    previous_id = get_previous_id(current_id)

    # Prepare the SQL query to retrieve the desired rows
    query = 'SELECT * FROM tests WHERE id IN (?, ?)'
    cursor.execute(query, (current_id, previous_id))

    # Fetch all matching rows
    items = cursor.fetchall()

    # Close the connection
    conn.close()

    return items

In [11]:
def map_items_to_args(items):
    # The file paths from the test x-1 are used as context
    html_path = items[0][3]
    image_path = items[0][4]
    precondition_path = items[0][5]
    # The last step of the text x is used as prompt
    steps = items[1][1].split(']')
    description = steps[-1].strip()

    return {"html_path": html_path,
            "image_path": image_path,
            "precondition_path": precondition_path,
            "description": description}

In [12]:
# Select test ID and database file
db_file = '../data/raw/playwright_script.db'
current_id = '1.4'

In [13]:
# Get relevant data from the database
items = fetch_relevant_items(db_file, current_id)
args = map_items_to_args(items)

In [14]:
items

[('1.3',
  '[1.1] Öffne die Arbeitsmappe "Übersicht Messstellen" im Ordner "Gewässergüte". [1.2]  Öffnen der Tabellen-Sicht "Messstellenliste" über die Werkzeugliste der Arbeitsmappe. [1.3] Klicke auf den 3-Punkte-Button innerhalb der Tabellen-Sicht "Messstelleninformationen"',
  '[1.3] Expected result: Kontextmenü erscheint mit den Optionen: - Duplizieren - In Arbeitsblatt duplizieren - Exportieren - Designer öffnen - Löschen.',
  '.\\html\\1_3.html',
  '.\\screenshot\\1_3.png',
  '.\\test_script\\1_3.spec.ts'),
 ('1.4',
  '[1.1] Öffne die Arbeitsmappe "Übersicht Messstellen" im Ordner "Gewässergüte". [1.2]  Öffnen der Tabellen-Sicht "Messstellenliste" über die Werkzeugliste der Arbeitsmappe. [1.3] Klicke auf den 3-Punkte-Button innerhalb der Tabellen-Sicht "Messstelleninformationen". [1.4] Klicke auf "Duplizieren".',
  '[1.4] Expected result: Das Kontextmenü schließt sich. Die Sicht ist dupliziert. Die duplizierte Sicht erscheint neben der Original-Sicht. Beide Sichten teilen sich in

In [15]:
args

{'html_path': '.\\html\\1_3.html',
 'image_path': '.\\screenshot\\1_3.png',
 'precondition_path': '.\\test_script\\1_3.spec.ts',
 'description': 'Klicke auf "Duplizieren".'}

2. Now we can run the main function to sum it up with the extracted information, we will get the generated UI test code for the test with id 1.4. Prompt engineering is implemented in the main function.

In [28]:
%%time
# Generate UI test code
prompt = main(**args)

2024-07-02 10:31:59 [src.main:27] [INFO] >>>> Loading context...
2024-07-02 10:32:01 [src.data.image_processor:27] [DEBUG] >>>> Text extracted from image successfully. - Characters: 303
2024-07-02 10:32:01 [src.data.code_processor:15] [DEBUG] >>>> Code parsed successfully. - Lines of Code: 17
2024-07-02 10:32:01 [src.main:31] [INFO] >>>> Context loaded successfully.
2024-07-02 10:32:01 [src.main:33] [INFO] >>>> Creating input prompt...
2024-07-02 10:32:01 [src.main:35] [INFO] >>>> Input prompt created successfully.
2024-07-02 10:32:01 [src.main:36] [DEBUG] >>>> Input prompt:
### Simplified HTML Content:
Buttons: 
{'id': 'navigationTrigger', 'class': 'button button-icon button-borderless'}, 
{'id': 'ad-hoc-settings-LO8EpTw_Rr6ParizyY3AtQ', 'class': 'button button-borderless button-xs is-hover-visible button-icon'}, 
{'id': 'Wus3-uWT-MicFD-66fN0t', 'class': 'button button-borderless button-s button-icon'}
Inputs: 
{'id': 'cpwGAhii7vkbtfgBbOaJ-', 'class': 'd-condition-date-picker--input f

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\merti\anaconda3\envs\uitest\Lib\logging\__init__.py", line 1160, in emit
    msg = self.format(record)
          ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\merti\anaconda3\envs\uitest\Lib\logging\__init__.py", line 999, in format
    return fmt.format(record)
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\merti\anaconda3\envs\uitest\Lib\logging\__init__.py", line 703, in format
    record.message = record.getMessage()
                     ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\merti\anaconda3\envs\uitest\Lib\logging\__init__.py", line 392, in getMessage
    msg = msg % self.args
          ~~~~^~~~~~~~~~~
TypeError: not all arguments converted during string formatting
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\merti\anaconda3\envs\uitest\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  F

# (Cluster) Pipeline: LLM UI Test Generation
Using Llava 1.5 model

1. Extract relevant information from the database for a specific test to be predicted:

In [17]:
# Select test ID and database file
db_file = '../data/raw/playwright_script.db'
current_id = '1.4'

In [18]:
items = fetch_relevant_items(db_file, current_id)
args = map_items_to_args(items)

In [19]:
items

[('1.3',
  '[1.1] Öffne die Arbeitsmappe "Übersicht Messstellen" im Ordner "Gewässergüte". [1.2]  Öffnen der Tabellen-Sicht "Messstellenliste" über die Werkzeugliste der Arbeitsmappe. [1.3] Klicke auf den 3-Punkte-Button innerhalb der Tabellen-Sicht "Messstelleninformationen"',
  '[1.3] Expected result: Kontextmenü erscheint mit den Optionen: - Duplizieren - In Arbeitsblatt duplizieren - Exportieren - Designer öffnen - Löschen.',
  '.\\html\\1_3.html',
  '.\\screenshot\\1_3.png',
  '.\\test_script\\1_3.spec.ts'),
 ('1.4',
  '[1.1] Öffne die Arbeitsmappe "Übersicht Messstellen" im Ordner "Gewässergüte". [1.2]  Öffnen der Tabellen-Sicht "Messstellenliste" über die Werkzeugliste der Arbeitsmappe. [1.3] Klicke auf den 3-Punkte-Button innerhalb der Tabellen-Sicht "Messstelleninformationen". [1.4] Klicke auf "Duplizieren".',
  '[1.4] Expected result: Das Kontextmenü schließt sich. Die Sicht ist dupliziert. Die duplizierte Sicht erscheint neben der Original-Sicht. Beide Sichten teilen sich in

In [20]:
# To traverse folders in cluster change path
args['image_path'] = args['image_path'].replace('\\', '/')

In [21]:
args

{'html_path': '.\\html\\1_3.html',
 'image_path': './screenshot/1_3.png',
 'precondition_path': '.\\test_script\\1_3.spec.ts',
 'description': 'Klicke auf "Duplizieren".'}

2. Setup access to the cluster:

Precondition:
- Adjust the paths in src file with your specific student credentials
- Make sure that playwright is installed on your local machine

In [24]:
access2cluster = Access2Cluster()
await access2cluster.login()

NotImplementedError: 

3. Init the Llava 1.5 model:

In [27]:
await access2cluster.start_llm()
args['model'] = access2cluster

In [28]:
args

{'html_path': '.\\html\\1_3.html',
 'image_path': '.\\screenshot\\1_3.png',
 'precondition_path': '.\\test_script\\1_3.spec.ts',
 'description': 'Klicke auf "Duplizieren".',
 'model': 'access2cluster'}

4. Run the main function to sum it up with the extracted information, we will get the generated UI test code for the test with id 1.4. Prompt engineering is implemented in the main function.

In [29]:
await main(**args)

2024-07-02 03:05:32 [src.main:29] [INFO] >>>> Loading context...


KeyError: 'id'

# More Developing: Using modular source code
Possible to update function content in the following files, but no RENAMING or DELETING or changing function signature!:
* src/main.py: Main function for UI test generation.
    * Change constant Max length (max length of the input text for the model)
* src/ui_tests/test_generation.py: Generate the UI test code using the language model.
    * Expand model selection: Just add new case statements for different models with its code.
* src/data/html_processing.py: Extract relevant information from the HTML file.
* src/data/image_processing.py: Extract relevant information from the image.
* src/data/python_processing.py: Parse the given playwright test code for previous step as a precondition.
